## Fix the auto-generated captions

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

True

### Load the Gemini model

In [21]:
from google import genai

client = genai.Client(api_key=os.getenv('GOOGLE_API_KEY'))

In [22]:
from pprint import pprint
for model in client.models.list():
    pprint({
        "name": model.name,
        "input_token_limit": model.input_token_limit,
        "output_token_limit": model.output_token_limit,
        "supported_actions": model.supported_actions
    })
    print()

{'input_token_limit': 4096,
 'name': 'models/chat-bison-001',
 'output_token_limit': 1024,
 'supported_actions': ['generateMessage', 'countMessageTokens']}

{'input_token_limit': 8196,
 'name': 'models/text-bison-001',
 'output_token_limit': 1024,
 'supported_actions': ['generateText',
                       'countTextTokens',
                       'createTunedTextModel']}

{'input_token_limit': 1024,
 'name': 'models/embedding-gecko-001',
 'output_token_limit': 1,
 'supported_actions': ['embedText', 'countTextTokens']}

{'input_token_limit': 12288,
 'name': 'models/gemini-1.0-pro-vision-latest',
 'output_token_limit': 4096,
 'supported_actions': ['generateContent', 'countTokens']}

{'input_token_limit': 12288,
 'name': 'models/gemini-pro-vision',
 'output_token_limit': 4096,
 'supported_actions': ['generateContent', 'countTokens']}

{'input_token_limit': 2000000,
 'name': 'models/gemini-1.5-pro-latest',
 'output_token_limit': 8192,
 'supported_actions': ['generateContent', 'countToke

### Use the model

First, set the system prompt.

In [30]:
system_prompt = """You are a native Arabic speaker with a strong command of the language. Your task is to review and refine YouTube captions that were automatically generated for Arabic videos. As you go through the captions, please correct any mistakes you find, as automated transcription may contain errors, but stay as close as possible to the original text. Don't use diacritics, you should provide the user with raw Arabic text.

Additionally, remove any placeholders such as [موسيقى], [تصفيق], [ضحك], and similar tags, keeping only the spoken text.

Reply to the user directly with the corrected and cleaned-up captions.
"""

Prepare the captions that you extracted with [get_captions_from_youtube_video](./get_captions_from_youtube_video.ipynb).

In [24]:
raw_captions = """
1
00:00:04,059 --> 00:00:15,839
[موسيقى]

2
00:00:23,240 --> 00:00:33,030
[موسيقى]

3
00:00:30,119 --> 00:01:02,169
ا

4
00:00:33,030 --> 00:01:02,169
[موسيقى]

5
00:01:10,170 --> 00:01:22,880
[موسيقى]

6
00:01:17,840 --> 00:01:24,520
اسمع ايها الانسان وشاهد وتمعن ايها

7
00:01:22,880 --> 00:01:28,680
الموصوف بالعقل

8
00:01:24,520 --> 00:01:30,799
والحكمه الحياه معجزه ظهرت منذ اربعه

9
00:01:28,680 --> 00:01:35,079
مليارات من الاعوام

10
00:01:30,799 --> 00:01:40,759
ونحن بنو الانسان لم يمضي على وجودنا سوى

11
00:01:35,079 --> 00:01:44,960
متي الف من الاعوام انها حقبه قصيره لكنها

12
00:01:40,759 --> 00:01:48,880
كانت كافيه لاختلال توازن ارضنا حتى باتت

13
00:01:44,960 --> 00:01:53,719
حياتنا على كوكبنا الازرق مهدده

14
00:01:48,880 --> 00:01:57,719
بالزوال استمعوا لهذه الحكايه حكايتنا

15
00:01:53,719 --> 00:01:57,719
حكايه الارض

16
00:02:03,220 --> 00:02:05,190
[تصفيق]

17
00:02:04,380 --> 00:02:08,310
[موسيقى]

18
00:02:05,190 --> 00:02:08,310
[تصفيق]

19
00:02:10,789 --> 00:02:18,480
[تصفيق]

20
00:02:14,000 --> 00:02:22,400
شاهدوا اثار نشات الاولى في البدايه لم

21
00:02:18,480 --> 00:02:27,480
يكن كوكبنا سوى كتله ناريه مضطربه تكونت

22
00:02:22,400 --> 00:02:32,879
من الشمس وانفصلت عنها سحابه من جزيئات

23
00:02:27,480 --> 00:02:37,200
غاريه متصله تسبح في ضء شانها شان العديد

24
00:02:32,879 --> 00:02:42,840
من المجموعات المماثله في الكون لكن هنا

25
00:02:37,200 --> 00:02:42,840
حدثت المعجزه معجزه الحياه

26
00:02:52,720 --> 00:03:03,419
[موسيقى]

27
00:03:17,440 --> 00:03:24,239
هذه الحياه التي نعيشها اليوم حياتنا ليست

28
00:03:21,040 --> 00:03:27,560
سوى حلقه من سلسله لا تحصى ولا تعد ولا

29
00:03:24,239 --> 00:03:31,760
تنتهي من الكائنات الحيه التي توارثت

30
00:03:27,560 --> 00:03:31,760
الارض على مدى اربعه مليارات عام

31
00:03:34,120 --> 00:03:42,120
وحتى اليوم ما زالت البراكين الجديده تشكل

32
00:03:37,920 --> 00:03:44,000
معالم ارضنا فنست عيد معها بعضا من مشهد

33
00:03:42,120 --> 00:03:49,599
الارض في ميلادها

34
00:03:44,000 --> 00:03:53,239
الاول صخور منصهره تفور من باطنها تصبح

35
00:03:49,599 --> 00:03:58,239
سلطه تتشقق تعتريها البثرات

36
00:03:53,239 --> 00:03:59,930
تنبسط في هيئه قشره نحيفه ثم تخمد ردحا من

37
00:03:58,239 --> 00:04:03,039
الزمان

38
00:03:59,930 --> 00:04:03,039
[موسيقى]

39
00:04:03,239 --> 00:04:11,480
تتصاعد سحب الدخان من جوف الارض رقيقه

40
00:04:06,879 --> 00:04:16,160
تسبح وتتلو في الاثير لتصبح شاهدا حيا على

41
00:04:11,480 --> 00:04:21,160
جو ذلك الزمان جو خال من الاكسجين مليء

42
00:04:16,160 --> 00:04:21,160
بثاني اكسيد الكربون ومثقل ببخار

43
00:04:22,240 --> 00:04:29,870
الماء كانت الارض فرنا

44
00:04:26,560 --> 00:04:29,870
[موسيقى]

45
00:04:30,479 --> 00:04:40,280
لكنها كانت موعوده بمستقبل مدهش خباه لها

46
00:04:35,800 --> 00:04:46,120
الماء تبعد الارض عن الشمس مسافه تحفظ لها

47
00:04:40,280 --> 00:04:51,759
الحياه فلا تبعد بعدا يجمد ولا تدنو دنوا

48
00:04:46,120 --> 00:04:54,520
يحرقها توازن عجيب حفظ لها مائها سائلا

49
00:04:51,759 --> 00:04:54,520
يجري في

50
00:04:55,400 --> 00:05:04,520
اوصالها تبخر الماء وتكاثف في السماء

51
00:05:00,520 --> 00:05:08,840
فانهمرت وظهرت

52
00:05:04,520 --> 00:05:14,360
الانهار جرت الانهار فشكلت بان سيابها

53
00:05:08,840 --> 00:05:18,280
معالم سطح الارض شقت قنواتها وحفرت

54
00:05:14,360 --> 00:05:23,560
الوديان تدفقت الانهار بمياهها حتى ادنى

55
00:05:18,280 --> 00:05:28,440
البقاع فتكونت المحيطات في طريقها انتزعت

56
00:05:23,560 --> 00:05:33,120
المعادن من الصخور وشيئا فشيئا اصبحت مياه

57
00:05:28,440 --> 00:05:33,120
المحيطات العذبه مثقله

58
00:05:35,280 --> 00:05:41,960
بالملح من اين اتينا واين قدحت اولى

59
00:05:40,080 --> 00:05:46,919
شرارات

60
00:05:41,960 --> 00:05:51,520
الحياه معجزه من معجزات الزمان ما نراه

61
00:05:46,919 --> 00:05:56,280
هنا هو البكتيريا القديمه اشكال بدائيه من

62
00:05:51,520 --> 00:06:01,160
الحياه ما زالت موجوده في ينابيع الكوكب

63
00:05:56,280 --> 00:06:03,160
الحاره تهب الينابيع الوانها وتعرف باسم

64
00:06:01,160 --> 00:06:06,880
البكتيريا

65
00:06:03,160 --> 00:06:10,319
البدائيه تقتات جميعها من الحراره النابعه

66
00:06:06,880 --> 00:06:14,960
من الارض مادا البكتيريا السانيه او

67
00:06:10,319 --> 00:06:18,120
الطحالب الخضراء المزرقه وحدها البكتيريا

68
00:06:14,960 --> 00:06:19,440
السانيه لها القدره على الاستداره نحو

69
00:06:18,120 --> 00:06:23,919
الشمس

70
00:06:19,440 --> 00:06:27,120
لتستمع منها هذه البكتيريا الدقيقه

71
00:06:23,919 --> 00:06:29,880
بالاضافه الى المليارات من نسلها غيرت

72
00:06:27,120 --> 00:06:33,160
مصير كوكبنا باسره

73
00:06:29,880 --> 00:06:36,960
جوه القاتل الى جو ينبض

74
00:06:33,160 --> 00:06:41,440
بالحياه ربما نتساءل ماذا حل بالكربون

75
00:06:36,960 --> 00:06:44,720
الذي كان يسمم الجو والاجابه هي انه مازال

76
00:06:41,440 --> 00:06:44,720
حبيسا في القشره

77
00:06:45,639 --> 00:06:54,160
الارضيه في قديم الزمان كان هذا الوادي

78
00:06:49,479 --> 00:06:58,560
بحرا زاخرا بكائنات مجهريه حيه انبتت

79
00:06:54,160 --> 00:07:03,639
لنفسها اصفا بامتصاص الكربون الذائب في

80
00:06:58,560 --> 00:07:07,160
الجو فلما نفقت غرقت اصداف في قاع البحر

81
00:07:03,639 --> 00:07:10,879
وكونت طبقات الجير البيضاء التي استنزفت

82
00:07:07,160 --> 00:07:17,759
الكربون من الجو فظهرت حياه اخرى

83
00:07:10,879 --> 00:07:17,759
ونمت الجو القاتل تحول الى جو حافل

84
00:07:21,199 --> 00:07:31,599
بالحياه الحياه هي التي غيرت جو

85
00:07:26,879 --> 00:07:36,000
الارض استلت النباتات غ ئها من طاقه الشمس

86
00:07:31,599 --> 00:07:37,000
التي مكنتها من كسر جزيئات الماء لتحصل

87
00:07:36,000 --> 00:07:41,840
على

88
00:07:37,000 --> 00:07:41,840
الاكسجين ملا الاكسجين

89
00:07:42,759 --> 00:07:54,560
الهواء دوره الماء الطبيعيه على ارضنا هي

90
00:07:47,400 --> 00:08:02,159
دوره في تجدد مستمر شلالات بخار الماء سحب

91
00:07:54,560 --> 00:08:07,479
امطار ينابيع انهار بحار محيطات وانهار

92
00:08:02,159 --> 00:08:12,440
جليديه حلقات في سلسله محكمه هكذا تحتفظ

93
00:08:07,479 --> 00:08:17,039
الارض بمائها ولا ينقص منه شيء فمن الماء

94
00:08:12,440 --> 00:08:19,960
ذاته شربت جميع الكائنات الحيه التي

95
00:08:17,039 --> 00:08:25,199
تعاقبت على

96
00:08:19,960 --> 00:08:30,039
الارض الماء امره يحير العقل فتاره نراهما

97
00:08:25,199 --> 00:08:33,440
ان جاريا وتاره اخرى بخارا وطور

98
00:08:30,039 --> 00:08:33,440
نجده جليدا

99
00:08:33,680 --> 00:08:43,640
صافيا في سيبيريا تتجمد اصطح البحيرات في

100
00:08:38,919 --> 00:08:48,519
الشتاء مختزنه خصائص الماء اذ

101
00:08:43,640 --> 00:08:53,720
يتجمد ولانها اخف من الماء تطفو الاسطح

102
00:08:48,519 --> 00:08:57,210
الجليديه ولا تغرق فتشكل ستارا واقيا من

103
00:08:53,720 --> 00:09:11,749
البرد تستمر الحياه تحته
"""

Now, create the user query.

In [35]:
query = f"""I have extracted captions from a YouTube video, and I need your help to merge them into a single, coherent sequence of text. Please correct any grammatical errors and improve sentence structure as needed, since automatic transcription can be unreliable. You have the full context, so feel free to adjust sentences for clarity and flow. Also, please answer directly with the corrected captions and don't use diacritics, I need just the raw Arabic text. Thanks in advance! Here are the captions for you to process:

{raw_captions}
"""

Generate from the model.

In [36]:
from google.genai.types import GenerateContentConfig

response = client.models.generate_content(
    model='gemini-2.0-pro-exp-02-05',
    contents=query,
    config=GenerateContentConfig(
        system_instruction=[
            system_prompt
        ]
    )
)

print(response.text)

اسمع أيها الإنسان وشاهد وتمعن أيها الموصوف بالعقل والحكمة. الحياة معجزة ظهرت منذ أربعة مليارات من الأعوام، ونحن بنو الإنسان لم يمض على وجودنا سوى مئتي ألف من الأعوام، إنها حقبة قصيرة لكنها كانت كافية لاختلال توازن أرضنا حتى باتت حياتنا على كوكبنا الأزرق مهددة بالزوال. استمعوا لهذه الحكاية، حكايتنا، حكاية الأرض.

شاهدوا آثار نشأتنا الأولى. في البداية لم يكن كوكبنا سوى كتلة نارية مضطربة تكونت من الشمس وانفصلت عنها سحابة من جزيئات غازية متصلة تسبح في ضوء، شأنها شأن العديد من المجموعات المماثلة في الكون، لكن هنا حدثت المعجزة، معجزة الحياة.

هذه الحياة التي نعيشها اليوم، حياتنا ليست سوى حلقة من سلسلة لا تحصى ولا تعد ولا تنتهي من الكائنات الحية التي توارثت الأرض على مدى أربعة مليارات عام. وحتى اليوم ما زالت البراكين الجديدة تشكل معالم أرضنا، فنستعيد معها بعضا من مشهد الأرض في ميلادها الأول. صخور منصهرة تفور من باطنها تصبح صلدة، تتشقق، تعتريها البثرات، تنبسط في هيئة قشرة نحيفة ثم تخمد ردحا من الزمان.

تتصاعد سحب الدخان من جوف الأرض رقيقة تسبح وتتلوى في الأثير لتصبح شاهدا حيا على جو ذلك الزمان